# Что сделано

* Создан докер контейнер
* сокращены функции ```Context.__repr__```, ```Context.pretty_repr```
* вынес конструкцию ```a if a is not None else b``` в функцию ```get_not_none(a,b)```
* В очередной раз поменял синтаксис из-за ```networkx```
* Начал удалять переменные ```y_pred_mean```, ```y_fact_mean``` - надо перенести их в разряд метрик

In [1]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os
import sys
BASE_PATH = os.path.realpath('..')
DATASETS_DIR = os.path.join(BASE_PATH, 'datasets')
LIB_DIR = os.path.join(BASE_PATH,'lib')
if LIB_DIR not in sys.path:
    sys.path.append(LIB_DIR)

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from importlib import reload

In [3]:
import fca_interp as fcai

In [320]:
from importlib import reload

In [321]:
mango_ds = pd.read_csv(os.path.join(DATASETS_DIR, 'mango.csv'))
mango_ds.index = mango_ds['title']
mango_ds = mango_ds.drop('title',1)
mango_ds

,color,firm,smooth,form,fruit
title,,,,,
apple,yellow,False,True,round,True
grapefruit,yellow,False,False,round,True
kiwi,green,False,False,oval,True
plum,blue,False,True,oval,True
toy cube,green,True,True,cubic,False
egg,white,True,True,oval,False
tennis ball,white,False,False,round,False
mango,green,False,True,oval,True


In [323]:
mango_bin_ds = mango_ds.copy()
for f in mango_bin_ds.columns:
    if mango_bin_ds[f].dtype!=bool:
        for v in mango_bin_ds[f].unique():
            mango_bin_ds[f"{f}__is__{v}"] = mango_bin_ds[f]==v
        mango_bin_ds = mango_bin_ds.drop(f,1)

In [386]:
fcai = reload(fcai)

In [387]:
cntx = fcai.Context(mango_bin_ds)
fm = fcai.FormalManager(cntx)
fm.construct_concepts()
fm.construct_lattice()

In [392]:
fig = fm.get_plotly_fig()

In [393]:
fig.show()

In [498]:
bank_ds = pd.read_csv(os.path.join(DATASETS_DIR, 'bank.csv'), sep=';')

for f in ['default','y','housing','loan']:
    bank_ds[f] = (bank_ds[f]=='yes').astype(int)
cat_feats = ['job','marital','education', 'default','housing','loan', 'contact','month','poutcome', ]
real_feats = ['age','balance','day','duration','campaign','pdays','previous']
bank_ds

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,0,-333,1,0,cellular,30,jul,329,5,-1,0,unknown,0
4517,57,self-employed,married,tertiary,1,-3313,1,1,unknown,9,may,153,1,-1,0,unknown,0
4518,57,technician,married,secondary,0,295,0,0,cellular,19,aug,151,11,-1,0,unknown,0
4519,28,blue-collar,married,secondary,0,1137,0,0,cellular,6,feb,129,4,211,3,other,0


In [500]:
thresholds = {'age':[20, 30, 40, 50, 60],
              'balance':[0, 10, 100, 1000, 10000],
              'day':[10,20,25],
              'duration':[0,60,300, 600, 900],
              'campaign':[1,2],
              'pdays':[200,400],
             }
cases = {'pdays':[-1],
        'previous':[0]}

In [504]:
fcai = reload(fcai)

In [505]:
bank_bin_ds, feat_orders = fcai.Context.binarize_ds(bank_ds, cat_feats, thresholds, cases)

In [506]:
feat_orders

[('age__geq__60',
  'age__geq__50',
  'age__geq__40',
  'age__geq__30',
  'age__geq__20'),
 ('age__lt__20', 'age__lt__30', 'age__lt__40', 'age__lt__50', 'age__lt__60'),
 ('balance__geq__10000',
  'balance__geq__1000',
  'balance__geq__100',
  'balance__geq__10',
  'balance__geq__0'),
 ('balance__lt__0',
  'balance__lt__10',
  'balance__lt__100',
  'balance__lt__1000',
  'balance__lt__10000'),
 ('day__geq__25', 'day__geq__20', 'day__geq__10'),
 ('day__lt__10', 'day__lt__20', 'day__lt__25'),
 ('duration__geq__900',
  'duration__geq__600',
  'duration__geq__300',
  'duration__geq__60',
  'duration__geq__0'),
 ('duration__lt__0',
  'duration__lt__60',
  'duration__lt__300',
  'duration__lt__600',
  'duration__lt__900'),
 ('campaign__geq__2', 'campaign__geq__1'),
 ('campaign__lt__1', 'campaign__lt__2'),
 ('pdays__geq__400', 'pdays__geq__200'),
 ('pdays__lt__200', 'pdays__lt__400')]

In [488]:
binarize_ds(bank_ds, cat_feats, thresholds, cases)[0].drop_duplicates().T.drop_duplicates()

(4341, 124)

In [413]:
bank_bin_ds = bank_ds[cat_feats]
for f in cat_feats:
    if bank_bin_ds[f].nunique()>2:
        for v in bank_bin_ds[f].unique():
            bank_bin_ds[f"{f}__is__{v}"] = bank_bin_ds[f]==v
        bank_bin_ds = bank_bin_ds.drop(f,1)
    else:
        bank_bin_ds[f] = bank_bin_ds[f].astype(bool)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [323]:
mango_bin_ds = mango_ds.copy()
for f in mango_bin_ds.columns:
    if mango_bin_ds[f].dtype!=bool:
        for v in mango_bin_ds[f].unique():
            mango_bin_ds[f"{f}__is__{v}"] = mango_bin_ds[f]==v
        mango_bin_ds = mango_bin_ds.drop(f,1)

In [386]:
fcai = reload(fcai)

In [387]:
cntx = fcai.Context(mango_bin_ds)
fm = fcai.FormalManager(cntx)
fm.construct_concepts()
fm.construct_lattice()

In [392]:
fig = fm.get_plotly_fig()

In [393]:
fig.show()